# Ambertools-OpenMM Protein-ligand complex MD Setup tutorial using Jupyter Notebook
This notebook is wrritten by quantaosun@gmail.com in Shanghai, China, 2021. This notebook took PDB ID 7L10, the protease of SARS-CoV-2 with an inhibitor, as an example, relative work was firstly published on ACS Cent. Sci. 2021, 7, 3, 467–475, by William L. Jorgensen, Yale University.

Ambertools (https://ambermd.org/AmberTools.php) and Openmm (https://openmm.org/)should be properly refer to if any simulation result would be used in any publications.

In [ ]:
#install openmm
!yes | conda install -c conda-forge openmm &> /dev/null
# install openmm-setup for protein fix and simulation configuration
!yes | conda install -c conda-forge openmm-setup 
#install open babel
!yes|conda install -c openbabel openbabel &> /dev/null
# install Amber tools
!yes|conda install -c conda-forge ambertools=21 compilers 
#install visualization
! pip install py3Dmol 

In [ ]:
import sys
sys.path.append('/home/sqt/anaconda3/envs/Amber/lib/python3.7/site-packages')

# Download your PDB

In [ ]:
PDB_ID = "7L10" #@param {type:"string"}
pdb = PDB_ID + ".pdb"
! wget https://files.rcsb.org/download/$pdb

# Fix your protein with PDBfixer implemented in openmm-setup, please save the final strcture as "output.pdb" !!!

In [ ]:
!openmm-setup
#Click the square pause button to force quit after finished.

# Go to H++, register and process the output.pdb structure, download top and crd. If you leave everything default, you don't need to change anything below.http://biophysics.cs.vt.edu/index.php

# It is assumed you have save the processed pdb with a name of "output.pdb", otherwise please modifiy commands below.

# Generate protein pdb

In [ ]:
!ambpdb -p 0.15_80_10_pH6.5_output.top -c 0.15_80_10_pH6.5_output.crd > 0.pdb

In [ ]:
!awk '$4=="XEY"' 7L10.pdb > ligand1.pdb 

# Add hydrogens to you ligand

In [ ]:
!obabel -ipdb ligand1.pdb -opdb -O ligand2.pdb -h

# Beautify pdb format

In [ ]:
!pdb4amber -i ligand2.pdb -o ligand3.pdb

# Generate ligand mol2 file

In [ ]:
!antechamber -fi pdb -i ligand3.pdb -fo mol2 -o ligand4.mol2 -c bcc -pf y

# Make sure you see "calculation completed", otherwise something may have been wrong

In [ ]:
!tail  sqm.out

# Generate ligand prepi file

In [ ]:
!antechamber -i ligand4.mol2 -fi mol2 -o ligand5.prepi -fo prepi -pf y

# Generate ligand frcmod file

In [ ]:
!parmchk2 -f prepi -i ligand5.prepi -o ligand6.frcmod

# Combine protein with ligand again

In [ ]:
!cat 0.pdb ligand3.pdb > complex.pdb

# (Not working properly for now) Beautify PDB format, but we might just not need this anyway, skip for now.

In [ ]:
!pdb4amber -i complex.pdb -o complex_clean.pdb

# Create and run a file called tleap.in with Ambertools

In [ ]:
com_file = open('tleap.in','w')
com_file.write('''
source leaprc.protein.ff14SB #Source leaprc file for ff14SB protein force field
source leaprc.gaff #Source leaprc file for gaff
source leaprc.water.tip3p #Source leaprc file for TIP3P water model
loadamberprep ligand5.prepi #Load the prepi file for the ligand
loadamberparams ligand6.frcmod #Load the additional frcmod file for ligand
mol = loadpdb complex.pdb #Load PDB file for protein-ligand complex
solvatebox mol TIP3PBOX 8 #Solvate the complex with a cubic water box
addions mol Cl- 0 #Add Cl- ions to neutralize the system
saveamberparm mol complex.prmtop complex.inpcrd #Save AMBER topology and coordinate files
quit #Quit tleap program
''')
com_file.close()
!tleap -s -f tleap.in > tleap.out

# Upload prmtop and inpcrd to openmm-setup, and start the simulation! If you have a proper GPU, select CUDA, otherwise select CPU.

In [ ]:
!openmm-setup
#Click the square pause button to force quit after finished.

# Alternatively, you could save the simulation input files and run it on a cloud or HPC based OpenMM.

# If all goes well, you should see something like below (CPU only).

In [ ]:
Building system...
Performing energy minimization...
Equilibrating...
Simulating...
#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"
0.1%	1000	-488259.139341292	298.57624202824957	0
0.2%	2000	-492082.04384769656	300.09275305741824	16.6
0.3%	3000	-494725.1462343714	302.3090730320394	16.6
0.4%	4000	-496075.06792005355	302.3939891905172	16.4
0.5%	5000	-498015.23323377833	300.135566206444	16.4
0.6%	6000	-498488.50188332796	298.5545999644654	16.3
0.7%	7000	-499572.9765659773	300.7757759109986	16.3
0.8%	8000	-499030.79245316726	297.2492234858693	16.2
0.9%	9000	-499823.2656625096	300.60293949805	16.2
1.0%	10000	-500134.98822511156	297.5748675220697	16.2
1.1%	11000	-499849.7045947542	300.5532217535929	16.2
1.2%	12000	-500542.2799043103	299.2530600741315	16.2